![](img/prototype.py.png)

---

"""
*What is this pattern about?
This patterns aims to reduce the number of classes required by an
application. Instead of relying on subclasses it creates objects by
copying a prototypical instance at run-time.

This is useful as it makes it easier to derive new kinds of objects,
when instances of the class have only a few different combinations of
state, and when instantiation is expensive.

*What does this example do?
When the number of prototypes in an application can vary, it can be
useful to keep a Dispatcher (aka, Registry or Manager). This allows
clients to query the Dispatcher for a prototype before cloning a new
instance.

Below provides an example of such Dispatcher, which contains three
copies of the prototype: 'default', 'objecta' and 'objectb'.

*TL;DR
Creates new object instances by cloning prototype.
"""

In [1]:

from __future__ import annotations

import ruamel.yaml
from typing import Any

yaml = ruamel.yaml.YAML()

In [2]:
class Prototype:
    def __init__(self, value: str = "default", **attrs: Any) -> None:
        self.value = value
        self.__dict__.update(attrs)

    def clone(self, **attrs: Any) -> Prototype:
        """Clone a prototype and update inner attributes dictionary"""
        # Python in Practice, Mark Summerfield
        # copy.deepcopy can be used instead of next line.
        obj = self.__class__(**self.__dict__)
        obj.__dict__.update(attrs)
        return obj

In [3]:
class PrototypeDispatcher:
    def __init__(self):
        self._objects = {}

    def get_objects(self) -> dict[str, Prototype]:
        """Get all objects"""
        return self._objects

    def register_object(self, name: str, obj: Prototype) -> None:
        """Register an object"""
        self._objects[name] = obj

    def unregister_object(self, name: str) -> None:
        """Unregister an object"""
        del self._objects[name]

In [4]:
def main() -> None:
    yaml.register_class(PrototypeDispatcher)
    yaml.register_class(Prototype)
    
    dispatcher = PrototypeDispatcher()
    prototype = Prototype()

    d = prototype.clone()
    a = prototype.clone(value='a-value', category='a')
    b = a.clone(value='b-value', is_checked=True)
    dispatcher.register_object('objecta', a)
    dispatcher.register_object('objectb', b)
    dispatcher.register_object('default', d)

    [{n: p.value} for n, p in dispatcher.get_objects().items()]
    # [{'objecta': 'a-value'}, {'objectb': 'b-value'}, {'default': 'default'}]

    print(b.category, b.is_checked)
    # a True

    yaml.dump([dispatcher], sys.stdout)
    # - !PrototypeDispatcher
    #   _objects:
    #     objecta: !Prototype
    #       value: a-value
    #       category: a
    #     objectb: !Prototype
    #       value: b-value
    #       category: a
    #       is_checked: true
    #     default: !Prototype
    #       value: default

In [5]:
main()

a True
- !PrototypeDispatcher
  _objects:
    objecta: !Prototype
      value: a-value
      category: a
    objectb: !Prototype
      value: b-value
      category: a
      is_checked: true
    default: !Prototype
      value: default
